# Pinecone Hybrid Search

>[Pinecone](https://docs.pinecone.io/docs/overview)是一个功能全面的向量数据库。

这个笔记将介绍如何使用一个底层使用Pinecone和Hybrid Search的检索器。

这个检索器的逻辑来自于[这个文档](https://docs.pinecone.io/docs/hybrid-search)

要使用Pinecone，您必须拥有API密钥和环境。
这里是[安装说明](https://docs.pinecone.io/docs/quickstart)。

In [ ]:
#!pip install pinecone-client pinecone-text

In [ ]:
import os
import getpass

os.environ['PINECONE_API_KEY'] = getpass.getpass('Pinecone API Key:')

In [75]:
from langchain.retrievers import PineconeHybridSearchRetriever

In [ ]:
os.environ['PINECONE_ENVIRONMENT'] = getpass.getpass('Pinecone Environment:')

我们想使用 `OpenAIEmbeddings`，所以我们必须获取OpenAI API密钥。

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

## 设置Pinecone

这部分只需要做一次。

注意：重要的是确保元数据中包含的保存文本的“上下文”字段没有被索引。目前，您需要显式指定您想要索引的字段。有关更多信息，请查看Pinecone的[文档](https://docs.pinecone.io/docs/manage-indexes#selective-metadata-indexing)。

In [76]:
import os
import pinecone

api_key = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"
# find environment next to your API key in the Pinecone console
env = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"

index_name = "langchain-pinecone-hybrid-search"

pinecone.init(api_key=api_key, enviroment=env)
pinecone.whoami()

WhoAmIResponse(username='load', user_label='label', projectname='load-test')

In [77]:
 # create the index
pinecone.create_index(
   name = index_name,
   dimension = 1536,  # dimensionality of dense model
   metric = "dotproduct",  # sparse values supported only for dotproduct
   pod_type = "s1",
   metadata_config={"indexed": []}  # see explaination above
)

现在它已经创建好了，我们可以使用它了

In [78]:
index = pinecone.Index(index_name)

## 获取嵌入和稀疏编码器

嵌入用于稠密向量，tokenizer用于稀疏向量

In [79]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

要将文本编码为稀疏值，您可以选择SPLADE或BM25。对于领域外的任务，我们建议使用BM25。

有关稀疏编码器的更多信息，您可以查看Pinecone-text库[文档](https://pinecone-io.github.io/pinecone-text/pinecone_text.html)。

In [80]:
from pinecone_text.sparse import BM25Encoder
# or from pinecone_text.sparse import SpladeEncoder if you wish to work with SPLADE

# use default tf-idf values
bm25_encoder = BM25Encoder().default()

上面的代码正在使用默认的tfid值。强烈建议将tf-idf值适合自己的语料库。您可以按照以下步骤执行：

```python
corpus = ["foo", "bar", "world", "hello"]

# 在您的语料库上拟合tf-idf值
bm25_encoder.fit(corpus)

# 将值存储到json文件中
bm25_encoder.dump("bm25_values.json")

# 加载到BM25Encoder对象中
bm25_encoder = BM25Encoder().load("bm25_values.json")
```

## 加载检索器

我们现在可以构建检索器了！

In [81]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)

## 添加文本（如果必要）

我们可以选择将文本添加到检索器中（如果它们还不存在于其中）

In [82]:
retriever.add_texts(["foo", "bar", "world", "hello"])

100%|██████████| 1/1 [00:02<00:00,  2.27s/it]


## 使用检索器

我们现在可以使用检索器！

In [83]:
result = retriever.get_relevant_documents("foo")

In [84]:
result[0]

Document(page_content='foo', metadata={})